# <font color=#c51b8a>VPOD 'Mine-n-Match':</font>
## <font color=#c994c7>Part 1 Objective</font> - Use Species Names from Microspectrophotemetry Data Sheet to Query NCBI for All Related Opsin Sequences  

In [28]:
import os
import re
import datetime
import time
import mysql
import mysql.connector
import pandas as pd
from deepBreaks.preprocessing import read_data
from Bio import Entrez, SeqIO

In [29]:
scp_file = 'c:/Users/safra/Documents/GitHub/visual-physiology-opsin-db/scripts_n_notebooks/vpod_ML_workflows/subtests/work_in_progress/mine_n_match/scp_raw.tsv'
scp_df = read_data(scp_file, seq_type = None, is_main=False)

In [30]:
scp_df.head()

,Genus,Species,phylum,CellType,CellSubType,LambdaMax,error,Chromophore,Stage,useable,expid,opsinid,refid,Notes,Unnamed: 15,isGenusInHetero,isSpeciesInHetero,isInHetero,isVert
maxid,,,,,,,,,,,,,,,,,,,
1,Acipenser,transmontanus,Chordata,rod,NaN,540.0,NaN,NaN,adult,1.0,NaN,463,30,partial cds but is rod opsin,NaN,NaN,NaN,0,1.0
2,Acipenser,transmontanus,Chordata,cone,NaN,464.0,NaN,NaN,adult,0.0,NaN,0,30,NaN,NaN,NaN,NaN,0,NaN
3,Acipenser,transmontanus,Chordata,cone,NaN,531.0,NaN,NaN,adult,0.0,NaN,0,30,NaN,NaN,NaN,NaN,0,NaN
4,Acipenser,transmontanus,Chordata,cone,NaN,605.0,NaN,NaN,adult,0.0,NaN,0,30,NaN,NaN,NaN,NaN,0,NaN
5,Acipenser,medirostris,Chordata,rod,NaN,540.0,NaN,NaN,adult,0.0,NaN,0,43,NaN,NaN,NaN,NaN,0,NaN


In [31]:
species_list = []
for sp, genus in zip(scp_df['Genus'], scp_df['Species']):
    full_species = f'{sp} {genus}'
    species_list.append(full_species)
len(species_list)

1570

In [32]:
unique_species_list = list(set(species_list)) 
len(unique_species_list)

469

In [33]:
# create a function that return the collection of all NCBI fetch result

def ncbi_fetch(email, term, ncbi_db = "nuccore", rettype = "gb", format = "genbank"):
  Entrez.email = email    # Always tell NCBI who you are
  handle = Entrez.esearch(db=ncbi_db,
                        term=term, 
                        api_key = "1efb120056e1cea873ba8d85d6692abd5d09") # using api key allows 10 fetch per second
  record = Entrez.read(handle)
  #print(record)
  full_res = int(record["Count"])
  
  handle_full = Entrez.esearch(db=ncbi_db,
                        term=term, 
                        api_key = "1efb120056e1cea873ba8d85d6692abd5d09",
                        retmax = full_res + 1)
  record_full = Entrez.read(handle_full)
  print(record_full)
  q_list = record_full["IdList"]

# create a list for all the sequence fetched from NCBI
  NCBI_seq =[]
  for x in q_list:
    # fetch result from previous search
    fet = Entrez.efetch(db=ncbi_db, id=x, rettype=rettype)
    seq = SeqIO.read(fet, format)
    fet.close()
    
    NCBI_seq.append(seq)

    time.sleep(0.5)
  
  return NCBI_seq
  

In [34]:
scp_query_list = []
for species in unique_species_list:
    NCBI_seq = ncbi_fetch("sethfrazer@ucsb.edu", 
                    f"{species}[ORGN] AND (opsin[All Fields] AND complete[All Fields] AND cds[All Fields] NOT voucher)",)
    scp_query_list.append(NCBI_seq)
#Find the field for taxonomy name
    

{'Count': '0', 'RetMax': '0', 'RetStart': '0', 'IdList': [], 'TranslationSet': [{'From': 'Alburnus alburnus[ORGN]', 'To': '"Alburnus alburnus"[Organism]'}], 'TranslationStack': [{'Term': '"Alburnus alburnus"[Organism]', 'Field': 'Organism', 'Count': '480', 'Explode': 'Y'}, {'Term': 'opsin[All Fields]', 'Field': 'All Fields', 'Count': '45669', 'Explode': 'N'}, {'Term': 'complete[All Fields]', 'Field': 'All Fields', 'Count': '63796847', 'Explode': 'N'}, 'AND', {'Term': 'cds[All Fields]', 'Field': 'All Fields', 'Count': '188753934', 'Explode': 'N'}, 'AND', {'Term': 'voucher[All Fields]', 'Field': 'All Fields', 'Count': '12767405', 'Explode': 'N'}, 'NOT', 'GROUP', 'AND'], 'QueryTranslation': '"Alburnus alburnus"[Organism] AND (opsin[All Fields] AND complete[All Fields] AND cds[All Fields] NOT voucher[All Fields])'}
{'Count': '0', 'RetMax': '0', 'RetStart': '0', 'IdList': [], 'TranslationSet': [{'From': 'Dendrochirus zebra[ORGN]', 'To': '"Dendrochirus zebra"[Organism]'}], 'TranslationStack'

In [36]:
# create empty lists
Accession = []
DNA = []
Genus = []
Species = []
gene_des = []
version = []
Protein = []
full_sp_names = []
# loop through the result list obtained from the NCBI search
# may take over 10 minutes
for query in scp_query_list:
    for seq in query:
        # get genus nd speceis name seperately
        spe_name = seq.annotations["organism"]
        g_s_name = spe_name.split()

        # get and append protein sequence
        if seq.features:
            for feature in seq.features:
                if feature.type == "CDS":
                    if "translation" in feature.qualifiers.keys():
                        pro_seq = feature.qualifiers['translation'][0]

        # attached them to lists
        Accession.append(str(seq.name))
        Genus.append(str(g_s_name[0]))
        Species.append(str(g_s_name[1]))
        full_sp_names.append(str(g_s_name[0]) + '_' + str(g_s_name[1]))
        gene_des.append(str(seq.description))
        version.append(str(seq.id))
        Protein.append(str(pro_seq))


In [37]:
# create a dataframe for the information
ncbi_q_op = pd.DataFrame(
    {'Accession': version,
     'Genus': Genus,
     'Species': Species,
     'full_sp_name': full_sp_names,
     'Protein': Protein,
     'gene_des': gene_des
    })

In [38]:
ncbi_q_op.head()

,Accession,Genus,Species,full_sp_name,Protein,gene_des
0,DQ088639.1,Pseudotropheus,sp.,Pseudotropheus_sp.,MRGNRPTEFPDDFWIPVPLDTNNITAFSPYLVPQDHLGSPMIFYSM...,Pseudotropheus sp. 'acei' putative violet sens...
1,DQ088636.1,Pseudotropheus,sp.,Pseudotropheus_sp.,MKGKRDMELPEDFWIPVSLDTNNITSLSPFLVPQDHLGDTATFYTM...,Pseudotropheus sp. 'acei' putative blue sensit...
2,DQ088633.1,Pseudotropheus,sp.,Pseudotropheus_sp.,MAWEGGIEPNGTEGKNFYIPMSNRTGIVRSPFEYTQYYLADPIFFK...,Pseudotropheus sp. 'acei' putative green sensi...
3,DQ088630.1,Pseudotropheus,sp.,Pseudotropheus_sp.,MAWDGGIEPNGTEGKNFYIPMSNRTGIVRSPFEYTQYYMVDPIIYK...,Pseudotropheus sp. 'acei' putative green sensi...
4,DQ088627.1,Pseudotropheus,sp.,Pseudotropheus_sp.,MAEEWGKQSFAARRYHEDSTRGSAFAYTNSNNTRDPFEGPNYHIAP...,Pseudotropheus sp. 'acei' putative red sensiti...


In [39]:
mnm_job_label = 'msp_data'
dt_label = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
report_dir = f'mnm_on_{mnm_job_label}_{dt_label}'
os.makedirs(report_dir)
ncbi_q_op.to_csv(path_or_buf=f"./{report_dir}/ncbi_msp_ops_query.csv", index=False)

In [40]:
ncbi_q_op.shape

(679, 6)

## <font color=#c994c7>Part 2 and Part 3 Objective</font> - Clean-up NCBI data and format it to query OPTICS to obtain lmax predictions then match the sequence to it's closest MSP value based on those predictions

We'll need to...

- Format all sequences to a FASTA file from the existing df
- Query OPTICS with all the sequence data [bootstrap enabled]
- Extract Predictions
- Match to closest MSP value species-by-species [will need a list of the unique species names] // Match to MaxId as the foreign key

In [1]:
import os
import time 
import datetime
import warnings
import pandas as pd
from deepBreaks.preprocessing import read_data

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

## <font color=#c994c7>Part 2: Clean-up NCBI Data and Format for OPTICS predictions</font>

In [10]:
# Loading mined seq data and cleaning it of all redundant entries (cases where Species and Sequence are redundant - we keep cases where identical sequence is present in different species) 
report_dir = 'mnm_on_msp_data_16_10_2024' #re-define the report directory if needed
ncbi_query_file = f"./{report_dir}/ncbi_msp_ops_query.csv"
ncbi_data = read_data(ncbi_query_file, seq_type = None, is_main=False)
ncbi_data['full_species'] = ncbi_data['Genus'] + '_' + ncbi_data['Species']
ncbi_data_filtered = ncbi_data.copy().drop_duplicates(subset=['full_species', 'Protein'])
ncbi_data_filtered.to_csv(path_or_buf=f"./{report_dir}/filtered_ncbi_msp_ops_query.csv", index=True)
ncbi_data_filtered.head()

,Genus,Species,full_sp_name,Protein,gene_des,full_species
Accession,,,,,,
DQ088639.1,Pseudotropheus,sp.,Pseudotropheus_sp.,MRGNRPTEFPDDFWIPVPLDTNNITAFSPYLVPQDHLGSPMIFYSM...,Pseudotropheus sp. 'acei' putative violet sens...,Pseudotropheus_sp.
DQ088636.1,Pseudotropheus,sp.,Pseudotropheus_sp.,MKGKRDMELPEDFWIPVSLDTNNITSLSPFLVPQDHLGDTATFYTM...,Pseudotropheus sp. 'acei' putative blue sensit...,Pseudotropheus_sp.
DQ088633.1,Pseudotropheus,sp.,Pseudotropheus_sp.,MAWEGGIEPNGTEGKNFYIPMSNRTGIVRSPFEYTQYYLADPIFFK...,Pseudotropheus sp. 'acei' putative green sensi...,Pseudotropheus_sp.
DQ088630.1,Pseudotropheus,sp.,Pseudotropheus_sp.,MAWDGGIEPNGTEGKNFYIPMSNRTGIVRSPFEYTQYYMVDPIIYK...,Pseudotropheus sp. 'acei' putative green sensi...,Pseudotropheus_sp.
DQ088627.1,Pseudotropheus,sp.,Pseudotropheus_sp.,MAEEWGKQSFAARRYHEDSTRGSAFAYTNSNNTRDPFEGPNYHIAP...,Pseudotropheus sp. 'acei' putative red sensiti...,Pseudotropheus_sp.


In [6]:
ncbi_data_filtered.shape

(543, 6)

In [8]:
fasta_file = f'{report_dir}/ncbi_mined_seqs.fasta'
with open(fasta_file, 'w') as f:
    for id, seq in zip(ncbi_data_filtered.index.to_list(), ncbi_data_filtered['Protein']):
        f.write(f'>{id}\n{seq}\n')

## <font color=#c994c7>Part 3: Match sequences to closest MSP entry based on species and OPTICS predictions</font>

In [12]:
report_dir = 'mnm_on_msp_data_16_10_2024' #re-define the report directory if needed
filtered_ncbi_query_file = f'./{report_dir}/filtered_ncbi_msp_ops_query.csv'
ncbi_data_filtered = read_data(filtered_ncbi_query_file, seq_type = None, is_main=False)

In [13]:
# Example use for OPTICS
# python optics_predictions.py -in msp_mined_seqs.fasta -rd mined_msp_seqs -out mined_seq_predictions.tsv -m wildtype -e aa_prop -b True -ir msp_mined_seq_blastp_report.tsv -r bovine -s False -bsv msp_bs_viz.pdf
predFileData = f'./{report_dir}/optics_on_mined_msp_seqs2_wt_2024-09-29_17-14-30/msp_predictions.tsv'
pred_df = read_data(predFileData, seq_type = None, is_main=False)
pred_df.head()

,Single_Prediction,Prediction_Means,Prediction_Medians,Prediction_Lower_Bounds,Prediction_Upper_Bounds,Std_Deviation,%Identity_Nearest_VPOD_Sequence
Names,,,,,,,
DQ088639.1,422.1,422.6,422.7,415.5,431.5,3.4,95.157
DQ088636.1,457.3,453.2,454.9,437.9,464.2,7.1,81.766
DQ088633.1,515.1,515.7,516.4,508.8,519.5,3.0,96.307
DQ088630.1,525.9,524.2,524.9,516.9,527.9,3.2,96.307
DQ088627.1,556.2,554.5,554.9,544.1,561.6,4.6,94.118


In [14]:
mnm_merged_df = pd.merge(ncbi_data_filtered, pred_df, left_index=True, right_index=True)
mnm_merged_df.head()

,Genus,Species,full_sp_name,Protein,gene_des,full_species,Single_Prediction,Prediction_Means,Prediction_Medians,Prediction_Lower_Bounds,Prediction_Upper_Bounds,Std_Deviation,%Identity_Nearest_VPOD_Sequence
DQ088639.1,Pseudotropheus,sp.,Pseudotropheus_sp.,MRGNRPTEFPDDFWIPVPLDTNNITAFSPYLVPQDHLGSPMIFYSM...,Pseudotropheus sp. 'acei' putative violet sens...,Pseudotropheus_sp.,422.1,422.6,422.7,415.5,431.5,3.4,95.157
DQ088636.1,Pseudotropheus,sp.,Pseudotropheus_sp.,MKGKRDMELPEDFWIPVSLDTNNITSLSPFLVPQDHLGDTATFYTM...,Pseudotropheus sp. 'acei' putative blue sensit...,Pseudotropheus_sp.,457.3,453.2,454.9,437.9,464.2,7.1,81.766
DQ088633.1,Pseudotropheus,sp.,Pseudotropheus_sp.,MAWEGGIEPNGTEGKNFYIPMSNRTGIVRSPFEYTQYYLADPIFFK...,Pseudotropheus sp. 'acei' putative green sensi...,Pseudotropheus_sp.,515.1,515.7,516.4,508.8,519.5,3.0,96.307
DQ088630.1,Pseudotropheus,sp.,Pseudotropheus_sp.,MAWDGGIEPNGTEGKNFYIPMSNRTGIVRSPFEYTQYYMVDPIIYK...,Pseudotropheus sp. 'acei' putative green sensi...,Pseudotropheus_sp.,525.9,524.2,524.9,516.9,527.9,3.2,96.307
DQ088627.1,Pseudotropheus,sp.,Pseudotropheus_sp.,MAEEWGKQSFAARRYHEDSTRGSAFAYTNSNNTRDPFEGPNYHIAP...,Pseudotropheus sp. 'acei' putative red sensiti...,Pseudotropheus_sp.,556.2,554.5,554.9,544.1,561.6,4.6,94.118


In [15]:
scp_file = f'./{report_dir}/scp_raw.tsv'
scp_df = read_data(scp_file, seq_type = None, is_main=False)
sp_list = []
og_sp_list = scp_df['Species'].to_list()
for sp in og_sp_list:
    sp_list.append(sp.split(' ')[0])
scp_df['full_species'] = scp_df['Genus'] + '_' + sp_list
scp_df.head()

,Genus,Species,phylum,CellType,CellSubType,LambdaMax,error,Chromophore,Stage,useable,expid,opsinid,refid,Notes,Unnamed: 15,isGenusInHetero,isSpeciesInHetero,isInHetero,isVert,full_species
maxid,,,,,,,,,,,,,,,,,,,,
1,Acipenser,transmontanus,Chordata,rod,NaN,540.0,NaN,NaN,adult,1.0,NaN,463,30,partial cds but is rod opsin,NaN,NaN,NaN,0,1.0,Acipenser_transmontanus
2,Acipenser,transmontanus,Chordata,cone,NaN,464.0,NaN,NaN,adult,0.0,NaN,0,30,NaN,NaN,NaN,NaN,0,NaN,Acipenser_transmontanus
3,Acipenser,transmontanus,Chordata,cone,NaN,531.0,NaN,NaN,adult,0.0,NaN,0,30,NaN,NaN,NaN,NaN,0,NaN,Acipenser_transmontanus
4,Acipenser,transmontanus,Chordata,cone,NaN,605.0,NaN,NaN,adult,0.0,NaN,0,30,NaN,NaN,NaN,NaN,0,NaN,Acipenser_transmontanus
5,Acipenser,medirostris,Chordata,rod,NaN,540.0,NaN,NaN,adult,0.0,NaN,0,43,NaN,NaN,NaN,NaN,0,NaN,Acipenser_medirostris


In [16]:
# Get unique species from predictions
unique_species = list(mnm_merged_df['full_species'].unique())
len(unique_species)

92

In [42]:
# Initialize a list to store the matched results
matched_results = []
i = 0
# Iterate through each species
for species in unique_species:
  scp_df_copy = scp_df.copy()
  # Filter predictions and measurements for the current species
  species_predictions = mnm_merged_df[mnm_merged_df['full_species'] == species]
  species_measurements = scp_df_copy[scp_df_copy['full_species'] == species]
  if species_measurements.shape[0] == 0:
    i+=1
    print(f'\nThis species is missing a match: {species}\n')
  else:
      #print(species)
    # Iterate through each prediction for the current species
    for _, prediction_row in species_predictions.iterrows():
      prediction_value = prediction_row['Prediction_Means']
      try:
        accession = prediction_row['Accession']
      except:
        accession = prediction_row.name

      # Calculate absolute differences between the prediction and all measurements for the species
      species_measurements.loc[:, 'abs_diff'] = (species_measurements['LambdaMax'] - prediction_value).abs()
      # Find the closest measurement (handling ties)
      closest_measurement_row = species_measurements.sort_values('abs_diff').iloc[0]
      min_diff = closest_measurement_row['abs_diff']
      
      existing_match_index = next((i for i, match in enumerate(matched_results) if match['Accession'] == accession), None)

      if (existing_match_index is not None): 
        if (min_diff < matched_results[existing_match_index]['abs_diff']):
          # If it exists, compare the absolute differences and keep the better match
          matched_results[existing_match_index].update({
              'prediction_value': prediction_value,
              'closest_measurement': closest_measurement_row['LambdaMax'],
              'abs_diff': min_diff,
              'max_id': closest_measurement_row.name
          })
      else:
          # If it doesn't exist, add the new match
          matched_results.append({
              'Accession': accession,
              'full_species': species,
              'prediction_value': prediction_value,
              'closest_measurement': closest_measurement_row['LambdaMax'],
              'abs_diff': min_diff,
              'max_id': closest_measurement_row.name
          })
        
# Create a new dataframe from the matched results
matched_df = pd.DataFrame(matched_results)

print(f'There were {i} unmatched species')
print(matched_df.head())


This species is missing a match: Pseudotropheus_sp.


This species is missing a match: Ostorhinchus_doederleini


This species is missing a match: Ostorhinchus_cyanosoma


This species is missing a match: Carlito_syrichta


This species is missing a match: Tharsalea_rubidus


This species is missing a match: Sardinops_melanosticta


This species is missing a match: Ostorhinchus_cookii


This species is missing a match: Maylandia_benetos


This species is missing a match: Bombus_impatiens


This species is missing a match: Zoramia_leptacanthus


This species is missing a match: Zoramia_viridiventer


This species is missing a match: Rhabdamia_gracilis


This species is missing a match: Psalidodon_fasciatus


This species is missing a match: synthetic_construct


This species is missing a match: Maylandia_barlowi


This species is missing a match: Cephalopachus_bancanus


This species is missing a match: Gekko_gecko


This species is missing a match: Nectamia_savayensis

There were 18 u

In [18]:
matched_df.shape

(470, 6)

In [21]:
matched_df.to_csv(path_or_buf=f"./{report_dir}/mnm_unfiltered_match_results.csv", index=True)

In [23]:
len(matched_df['Accession'].unique())

470

In [25]:
len(matched_df['max_id'].unique())

186

In [43]:
#Match %Idenitities to Accessions
iden_list = []
prot_des_list = []
aa_seq_list = []
genus_list = []
species_list = []
for _, d in matched_df.iterrows():
    #print(d)
    acc = d['Accession']
    iden_list.append(mnm_merged_df.loc[acc]['%Identity_Nearest_VPOD_Sequence'])
    prot_des_list.append(mnm_merged_df.loc[acc]['gene_des'])
    aa_seq_list.append(mnm_merged_df.loc[acc]['Protein'])
    genus_list.append(mnm_merged_df.loc[acc]['Genus'])
    species_list.append(mnm_merged_df.loc[acc]['Species'])

matched_df['%Identity_Nearest_VPOD_Sequence'] = iden_list
matched_df['Gene_Description'] = prot_des_list
matched_df['Protein'] = aa_seq_list
matched_df['Genus'] = genus_list
matched_df['Species'] = species_list
match_id_list = range(len(aa_seq_list))
matched_df['mnm_id'] = match_id_list
matched_df = matched_df.reindex(columns=['mnm_id','Accession','Genus','Species','%Identity_Nearest_VPOD_Sequence','prediction_value','closest_measurement','abs_diff','max_id','Protein','Gene_Description','Notes'])
matched_df.set_index('mnm_id', inplace=True)

In [44]:
matched_df.head()

,Accession,Genus,Species,%Identity_Nearest_VPOD_Sequence,prediction_value,closest_measurement,abs_diff,max_id,Protein,Gene_Description,Notes
mnm_id,,,,,,,,,,,
0,MN519163.1,Aptychotrema,vincentiana,82.74,541.3,557.0,15.7,1166,MTESWNLVGYAARRRYEEEDTTRGSIFTYTNTNATRGPFEGPNYHI...,Aptychotrema vincentiana isolate 28777-1-1 lon...,NaN
1,MN519152.1,Aptychotrema,vincentiana,79.155,486.4,500.0,13.6,1163,MNGTEGENFYIPFPNKTGVVRSPYEFPQYYLAEPWKFSIIAAYMFL...,Aptychotrema vincentiana isolate 31915-1-1 gre...,NaN
2,MN519142.1,Aptychotrema,vincentiana,90.96,495.7,500.0,4.3,1163,MNGTEGDNFYVPMSNKTGIVRSPFEYPQYYLAEPWKFSALAAYMFF...,Aptychotrema vincentiana isolate 46583-1-1 rho...,NaN
3,JF823570.1,Poecilia,formosa,87.32,519.6,504.0,15.6,199,MAWDGGYEPNGTEGKNFYIPMSNKTGIVRSPFEYPQYYMVDPMIYK...,Poecilia formosa opsin M long-wave sensitive o...,NaN
4,JF823569.1,Poecilia,formosa,97.753,556.9,560.0,3.1,203,MAEEWGKQVFAARRHEDTTRGAAFTYTNSNHTKDPFEGPNYHIAPR...,Poecilia formosa opsin locus 1-L long-wave sen...,NaN


Once we have the dataframe of matched seqs - each accession is matched to an id - but some IDs may have several matches... 
At this point we do another 2 step filter

1. Filter by abs. error - keeping the accession with the lowest abs. error
2. If there is a tie - then filter by percent identitiy to sequences in VPOD 

In [45]:
import pandas as pd

# Group by 'max_id' and count unique accessions
grouped_counts = matched_df.groupby('max_id')['Accession'].nunique()

# Filter groups with more than one unique accession
duplicate_max_id_groups = grouped_counts[grouped_counts > 1]

if not duplicate_max_id_groups.empty:
    filtered_results = []

    # Iterate through each max_id with duplicates
    for max_id in duplicate_max_id_groups.index:
        # Filter rows with the current max_id
        duplicates = matched_df[matched_df['max_id'] == max_id]

        # Sort by abs_diff (ascending) and then percent_identity (descending)
        duplicates = duplicates.sort_values(['abs_diff', 'Accession'], ascending=[True, False])
        duplicates = duplicates.sort_values(['abs_diff', '%Identity_Nearest_VPOD_Sequence'], ascending=[True, False])

        # Keep only the first row (lowest abs_diff, highest percent_identity if tied)
        filtered_results.append(duplicates.iloc[0])

    # Combine filtered results with non-duplicate rows
    non_duplicates = matched_df[~matched_df['max_id'].isin(duplicate_max_id_groups.index)]
    final_filtered_df = pd.concat([pd.DataFrame(filtered_results), non_duplicates], ignore_index=True)
    final_filtered_df = final_filtered_df.sort_values(['abs_diff', '%Identity_Nearest_VPOD_Sequence'], ascending=[True, False])
    final_filtered_df.reset_index(drop=True, inplace=True)
else:
    final_filtered_df = matched_df 

final_filtered_df.head()

,Accession,Genus,Species,%Identity_Nearest_VPOD_Sequence,prediction_value,closest_measurement,abs_diff,max_id,Protein,Gene_Description,Notes
0,DQ168661.1,Poecilia,reticulata,98.034,560.0,560.0,0.0,1274,MAEEWGKQVFAARRHEDTTRGAAFTYTNSNHTKDPFEGPNYHIAPR...,Poecilia reticulata LWS_QUEm5_L06 long wave-se...,NaN
1,DQ088628.1,Melanochromis,vermivorus,93.838,555.0,555.0,0.0,389,MAEEWGKQSFAARRYHEDSTRGSAFAYTNSNNTRDPFEGPNYHIAP...,Melanochromis vermivorus putative red sensitiv...,NaN
2,L11865.1,Carassius,auratus,90.544,509.0,509.0,0.0,1349,MNGTEGKNFYVPMSNRTGLVRSPFEYPQYYLAEPWQFKILALYLFF...,"Carassius auratus green cone opsin mRNA, compl...",NaN
3,AY745192.1,Setonix,brachyurus,86.686,538.0,538.0,0.0,1475,MTQAWDPAGFLAWRRDENEETTRASLFVYTNSNNTKGPFEGPNYHI...,Setonix brachyurus long-wave sensitive opsin m...,NaN
4,MW219662.1,Myripristis,berndti,98.281,495.1,495.0,0.1,67,MNGTEGPYFYIPMSNATGVVRSPYEYPQYYLVYPAAFAVLGAYMFF...,"Myripristis berndti rhodopsin RH1 mRNA, comple...",NaN


In [46]:
final_filtered_df.shape

(186, 11)

In [48]:
final_filtered_df.to_csv(path_or_buf=f"./{report_dir}/mnm_results_id_filtered.csv", index=True)

In [49]:
#note that prediction values from optics are taken from the mean of the bootstrap predictions
final_err_filtered_df = final_filtered_df[final_filtered_df['abs_diff'] <= 15]
final_err_filtered_df = final_err_filtered_df[final_err_filtered_df['%Identity_Nearest_VPOD_Sequence'] != 100.000]

In [50]:
final_err_filtered_df.shape

(144, 11)

In [51]:
final_err_filtered_df.to_csv(path_or_buf=f"./{report_dir}/mnm_final_results_err_filtered.csv", index=True)